In [1]:
import pandas as pd
import os

In [2]:
dir_path = os.getcwd()
merge_data_path = f"{dir_path}/merge_data.xlsx"
merge_data_df = pd.read_excel(merge_data_path, sheet_name="merge_data")
merge_data_df.head()

,id_task,subject_task,description_task,sprint_id_task,sprint_task,sprint_estimated_start_task,sprint_estimated_finish_task,owner_task,owner_full_name_task,assigned_to_task,...,client_requirement_epic,team_requirement_epic,attachments,tags,watchers,voters,created_date,modified_date,epic_id,epic_name
0,3,task,NaN,1,sprint1,2024-02-21,2024-03-06,admin,admin,admin,...,False,False,0,NaN,[],0,2024-02-21 08:55:07.687307+00:00,2024-02-21 08:55:07.688703+00:00,11,location
1,4,admin_task_1,NaN,1,sprint1,2024-02-21,2024-03-06,admin,admin,admin,...,False,False,0,NaN,[],0,2024-02-21 08:55:07.687307+00:00,2024-02-21 08:55:07.688703+00:00,11,location
2,5,wisit_task_1,NaN,1,sprint1,2024-02-21,2024-03-06,admin,admin,wisit,...,False,False,0,NaN,[],0,2024-02-21 08:55:07.687307+00:00,2024-02-21 08:55:07.688703+00:00,11,location
3,6,admin_task_2,NaN,1,sprint1,2024-02-21,2024-03-06,admin,admin,admin,...,False,False,0,NaN,[],0,2024-02-21 06:41:55.288362+00:00,2024-02-21 06:42:18.975346+00:00,5,test_epic
4,7,wisit_task_2,NaN,1,sprint1,2024-02-21,2024-03-06,admin,admin,wisit,...,False,False,0,NaN,[],0,2024-02-21 06:41:55.288362+00:00,2024-02-21 06:42:18.975346+00:00,5,test_epic


In [3]:
assignee_list = merge_data_df["assigned_to_task"].unique().tolist()
assignee_list

['admin', 'wisit']

In [4]:
epic_name_list = merge_data_df["epic_name"].unique().tolist()
epic_name_list

['location', 'test_epic']

In [5]:
min_date = merge_data_df["start date"].min()
max_date = merge_data_df["end date"].max()
min_date, max_date

('2024-01-24', '2024-02-29')

In [6]:
def validate_start_date_end_date(date, holiday_list):
    # if start_date is weekend, then change to next Monday
    if date.weekday() in [5, 6]:
        date = date + pd.Timedelta(days=(7 - date.weekday()))
    while date in holiday_list:
        date = date + pd.Timedelta(days=1)
    return date


def get_holiday_list():
    holiday_list = [
        "2024/01/01",
        "2024/02/12",
        "2024/02/26",
        "2024/04/08",
        "2024/04/15",
        "2024/04/16",
        "2024/05/01",
        "2024/05/06",
        "2024/05/22",
        "2024/06/03",
        "2024/07/22",
        "2024/07/29",
        "2024/08/12",
        "2024/10/14",
        "2024/10/23",
        "2024/12/05",
        "2024/12/10",
        "2024/12/31",
    ]
    holiday_list = [pd.to_datetime(date) for date in holiday_list]
    # holiday_list = []
    return holiday_list


def get_work_day(start_date, end_date, holiday_list=[]):
    # count when it is not weekend
    date_range = pd.date_range(start=start_date, end=end_date)
    work_day = 0
    for date in date_range:
        if date in holiday_list:
            continue
        if date.weekday() in [5, 6]:
            continue
        work_day += 1
    if work_day == 0:
        work_day = 1
    return work_day


start_date = "15/01/2024"
end_date = "14/01/2024"
holiday_list = get_holiday_list()
start_date = pd.to_datetime(start_date, format="%d/%m/%Y")
end_date = pd.to_datetime(end_date, format="%d/%m/%Y")
get_work_day(
    start_date,
    end_date,
    holiday_list,
)

1

In [7]:
type(start_date)

pandas._libs.tslibs.timestamps.Timestamp

In [8]:
# from merge_data_df get actual time and start date, end date
task_daily_list = []

holiday_list = get_holiday_list()
for index, row in merge_data_df.iterrows():
    start_date = pd.to_datetime(row["start date"])
    end_date = pd.to_datetime(row["end date"])
    start_date = validate_start_date_end_date(start_date, holiday_list)
    end_date = validate_start_date_end_date(end_date, holiday_list)
    if end_date < start_date:
        end_date = start_date
    assignee = row["assigned_to_task"]
    epic_name = row["epic_name"]
    task_id = row["task_id"]
    actual_time = row["actual time"]
    total_date = get_work_day(start_date, end_date, holiday_list)
    hour = actual_time / total_date
    date_range = pd.date_range(start=start_date, end=end_date)
    for date in date_range:
        if date.weekday() in [5, 6]:
            continue
        if date in holiday_list:
            continue
        task_daily_list.append(
            {
                "date": date,
                "assignee": assignee,
                "epic_name": epic_name,
                "task_id": task_id,
                "hour": hour,
            }
        )
task_daily_df = pd.DataFrame(task_daily_list)
print(sum(task_daily_df["hour"]))
task_daily_df

318.99999999999994


,date,assignee,epic_name,task_id,hour
0,2024-01-24,admin,location,10,0.333333
1,2024-01-25,admin,location,10,0.333333
2,2024-01-26,admin,location,10,0.333333
3,2024-01-29,admin,location,10,0.333333
4,2024-01-30,admin,location,10,0.333333
5,2024-01-31,admin,location,10,0.333333
6,2024-02-15,admin,location,12,2.000000
7,2024-02-16,admin,location,12,2.000000
8,2024-02-19,admin,location,12,2.000000
9,2024-02-20,admin,location,12,2.000000


In [9]:
assignee_hour_per_day_df = (
    task_daily_df.groupby(["date", "assignee"]).agg({"hour": "sum"}).reset_index()
)
print(sum(assignee_hour_per_day_df["hour"]))
# pivot table
assignee_hour_per_day_df = assignee_hour_per_day_df.pivot(
    index="date", columns="assignee", values="hour"
)
# fill na with 0
assignee_hour_per_day_df = assignee_hour_per_day_df.fillna(0)
min_date = assignee_hour_per_day_df.index.min()
max_date = assignee_hour_per_day_df.index.max()
# fill missing date
date_range = pd.date_range(start=min_date, end=max_date)
assignee_hour_per_day_df = assignee_hour_per_day_df.reindex(date_range, fill_value=0)
# change column name first column to 'date'
assignee_hour_per_day_df = assignee_hour_per_day_df.reset_index()
assignee_hour_per_day_df = assignee_hour_per_day_df.rename(columns={"index": "date"})
# remove first column
assignee_hour_per_day_df

318.99999999999994


assignee,date,admin,wisit
0,2024-01-24,0.333333,0.000000
1,2024-01-25,0.333333,0.000000
2,2024-01-26,0.333333,0.000000
3,2024-01-27,0.000000,0.000000
4,2024-01-28,0.000000,0.000000
5,2024-01-29,0.333333,0.000000
6,2024-01-30,0.333333,0.000000
7,2024-01-31,0.333333,0.000000
8,2024-02-01,0.000000,0.000000
9,2024-02-02,0.000000,0.000000


In [10]:
project_hour_per_day_df = (
    task_daily_df.groupby(["date", "epic_name"]).agg({"hour": "sum"}).reset_index()
)
project_hour_per_day_df = project_hour_per_day_df.pivot(
    index="date", columns="epic_name", values="hour"
)
project_hour_per_day_df = project_hour_per_day_df.fillna(0)
min_date = project_hour_per_day_df.index.min()
max_date = project_hour_per_day_df.index.max()
date_range = pd.date_range(start=min_date, end=max_date)
project_hour_per_day_df = project_hour_per_day_df.reindex(date_range, fill_value=0)
project_hour_per_day_df = project_hour_per_day_df.reset_index()
project_hour_per_day_df = project_hour_per_day_df.rename(columns={"index": "date"})
project_hour_per_day_df

epic_name,date,location,test_epic
0,2024-01-24,0.333333,0.000000
1,2024-01-25,0.333333,0.000000
2,2024-01-26,0.333333,0.000000
3,2024-01-27,0.000000,0.000000
4,2024-01-28,0.000000,0.000000
5,2024-01-29,0.333333,0.000000
6,2024-01-30,0.333333,0.000000
7,2024-01-31,0.333333,0.000000
8,2024-02-01,0.000000,0.000000
9,2024-02-02,0.000000,0.000000


In [11]:
# save to excel

with pd.ExcelWriter("task_daily_df.xlsx") as writer:
    merge_data_df.to_excel(writer, sheet_name="merge_data_df")
    task_daily_df.to_excel(writer, sheet_name="task_daily_df")
    assignee_hour_per_day_df.to_excel(writer, sheet_name="assignee_hour_per_day_df")
    project_hour_per_day_df.to_excel(writer, sheet_name="project_hour_per_day_df")

In [12]:
# create plotly chart for weekly report
import plotly.express as px
import plotly.graph_objects as go

In [13]:
print(assignee_hour_per_day_df)

assignee       date      admin      wisit
0        2024-01-24   0.333333   0.000000
1        2024-01-25   0.333333   0.000000
2        2024-01-26   0.333333   0.000000
3        2024-01-27   0.000000   0.000000
4        2024-01-28   0.000000   0.000000
5        2024-01-29   0.333333   0.000000
6        2024-01-30   0.333333   0.000000
7        2024-01-31   0.333333   0.000000
8        2024-02-01   0.000000   0.000000
9        2024-02-02   0.000000   0.000000
10       2024-02-03   0.000000   0.000000
11       2024-02-04   0.000000   0.000000
12       2024-02-05   0.000000   0.000000
13       2024-02-06   0.000000   0.000000
14       2024-02-07   0.000000   0.000000
15       2024-02-08  10.000000   0.000000
16       2024-02-09  10.000000   0.000000
17       2024-02-10   0.000000   0.000000
18       2024-02-11   0.000000   0.000000
19       2024-02-12   0.000000   0.000000
20       2024-02-13  10.000000   3.750000
21       2024-02-14  10.000000   3.750000
22       2024-02-15  23.555556   3

In [14]:
from typing import List, Dict

# import make_subplots
from plotly.subplots import make_subplots


def creat_fully_report_graph(
    num_date_dict: Dict[str, int],
    pivot_hour_per_day_df: pd.DataFrame,
    fig_title: str = "Fully Report",
) -> go.Figure:
    num_date_list = list(num_date_dict.values())
    column_list = pivot_hour_per_day_df.columns.tolist()
    # remove date
    column_list.remove("date")
    print("column_list", column_list)
    fig = make_subplots(
        rows=len(num_date_list),
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.15,
    )
    showlegend = True
    # create color dict  from  column_list
    color_line_dict = {
        column: px.colors.qualitative.Plotly[i] for i, column in enumerate(column_list)
    }
    for plot_ind, (plot_name, num_date) in enumerate(num_date_dict.items()):
        if plot_ind != 0:
            showlegend = False
        report_plot = pivot_hour_per_day_df.tail(num_date)
        report_plot = report_plot.reset_index()
        first_date = report_plot["date"].min()
        last_date = report_plot["date"].max()
        work_hour_list = [8] * len(report_plot)
        date_list = report_plot["date"].tolist()
        for date in date_list:
            if date in holiday_list:
                work_hour_list[date_list.index(date)] = 0
            if date.weekday() in [5, 6]:
                work_hour_list[date_list.index(date)] = 0

        for column in column_list:
            fig.add_trace(
                go.Scatter(
                    x=report_plot["date"],
                    y=report_plot[column],
                    mode="lines+markers",
                    name=column,
                    showlegend=showlegend,
                    legendgroup=column,
                    line=dict(color=color_line_dict[column]),
                ),
                row=plot_ind + 1,
                col=1,
            )

        fig.add_trace(
            go.Scatter(
                x=report_plot["date"],
                y=work_hour_list,
                mode="lines",
                name="8 hour",
                line=dict(color="green", dash="dash"),
                showlegend=False,
            ),
            row=plot_ind + 1,
            col=1,
        )

        fig.update_layout(
            xaxis=dict(
                tickmode="array",
                tickvals=report_plot["date"].tolist(),
                ticktext=report_plot["date"].dt.strftime("%d-%b").tolist(),
                tickangle=45,
            ),
        )
        fig.update_xaxes(
            **dict(
                tickmode="array",
                tickvals=report_plot["date"].tolist(),
                ticktext=report_plot["date"].dt.strftime("%d-%b").tolist(),
                tickangle=45,
            ),
            row=plot_ind + 1,
            col=1,
            title_text=plot_name,
        )

        fig.update_yaxes(title_text="Hour", row=plot_ind + 1, col=1)
        fig.update_layout(
            title=fig_title,
            title_x=0.5,
            # title font size
            title_font_size=30,
        )
        #     showlegend=True,  # Show the legend
        #     legend=dict(
        #         x=0.5,
        #         y=0.5,
        #         orientation="h",
        #     ),
        # )
    return fig


num_date_dict = {
    "Bi-Weekly": 14,
    "Monthly": 30,
    "Quarterly": 90,
}

fully_report_graph = creat_fully_report_graph(
    num_date_dict,
    assignee_hour_per_day_df,
    fig_title="Assignee Hour Per Day Report",
)
fully_report_graph.show()
fully_report_graph.write_html("Assignee Hour Per Day Report.html")

column_list ['admin', 'wisit']


In [15]:
num_date_dict = {
    "Bi-Weekly": 14,
    "Monthly": 30,
    "Quarterly": 90,
}

fully_report_graph = creat_fully_report_graph(
    num_date_dict,
    project_hour_per_day_df,
    fig_title="Project Hour Per Day Report",
)
fully_report_graph.show()
fully_report_graph.write_html("Project Hour Per Day Report.html")

column_list ['location', 'test_epic']
